In [1]:
from IPython.core.display import HTML
HTML("""
<style>
.container {
    width: 100%;
}
</style>
""")

In [2]:
import graphlab as gl
gl.canvas.set_target('ipynb')
import tools 
import initialise

2016-03-15 04:58:58,920 [INFO] graphlab.cython.cy_server, 176: GraphLab Create v1.8.4 started. Logging: /tmp/graphlab_server_1458017937.log


This non-commercial license of GraphLab Create is assigned to kevin.mcisaac@gmail.com and will expire on November 06, 2016. For commercial licensing options, visit https://dato.com/buy/.


In [3]:
train_data, test_data = initialise.load_data(reload_data=False)
folds = gl.cross_validation.KFold(train_data, 5)

Loading saved processed data
284  features in total


In [4]:
params = {'target':'TARGET', 'random_seed':8923,
          'class_weights':'auto','metric':'auc',
          'early_stopping_rounds':20, 'max_iterations':500,
          'step_size': 0.1,
          'max_depth': 5,
          'column_subsample': 0.6}

In [5]:
job = gl.grid_search.create(folds,gl.boosted_trees_classifier.create,
                             params)
results = job.get_results().sort('mean_validation_accuracy', ascending=False)
results

2016-03-15 04:59:01,318 [INFO] graphlab.deploy.job, 22: Validating job.
2016-03-15 04:59:01,326 [INFO] graphlab.deploy.job, 36: Creating a LocalAsync environment called 'async'.
2016-03-15 04:59:01,332 [INFO] graphlab.deploy.map_job, 186: Validation complete. Job: 'Model-Parameter-Search-Mar-15-2016-04-59-0100000' ready for execution
2016-03-15 04:59:01,446 [INFO] graphlab.deploy.map_job, 192: Job: 'Model-Parameter-Search-Mar-15-2016-04-59-0100000' scheduled.
2016-03-15 04:59:13,605 [INFO] graphlab.deploy.job, 22: Validating job.
2016-03-15 04:59:13,609 [INFO] graphlab.deploy.map_job, 220: A job with name 'Model-Parameter-Search-Mar-15-2016-04-59-0100000' already exists. Renaming the job to 'Model-Parameter-Search-Mar-15-2016-04-59-0100000-17353'.
2016-03-15 04:59:13,614 [INFO] graphlab.deploy.map_job, 186: Validation complete. Job: 'Model-Parameter-Search-Mar-15-2016-04-59-0100000-17353' ready for execution
2016-03-15 04:59:13,726 [INFO] graphlab.deploy.map_job, 192: Job: 'Model-Param

Columns:
	class_weights	str
	column_subsample	float
	early_stopping_rounds	int
	max_depth	int
	max_iterations	int
	metric	str
	random_seed	int
	step_size	float
	target	str
	num_folds	int
	fold_id	list
	mean_training_accuracy	float
	mean_validation_accuracy	float
	model_id	list

Rows: 1

Data:
+---------------+------------------+-----------------------+-----------+----------------+
| class_weights | column_subsample | early_stopping_rounds | max_depth | max_iterations |
+---------------+------------------+-----------------------+-----------+----------------+
|      auto     |       0.6        |           20          |     5     |      500       |
+---------------+------------------+-----------------------+-----------+----------------+
+--------+-------------+-----------+--------+-----------+-----------------+
| metric | random_seed | step_size | target | num_folds |     fold_id     |
+--------+-------------+-----------+--------+-----------+-----------------+
|  auc   |     8923    |    0.1    | TARGET |     5     | [3, 1, 4, 0, 2] |
+--------+-------------+-----------+--------+-----------+-----------------+
+------------------------+--------------------------+-----------------+
| mean_training_accuracy | mean_validation_accuracy |     model_id    |
+------------------------+--------------------------+-----------------+
|     0.791436464088     |      0.78608260984       | [3, 1, 4, 0, 2] |
+------------------------+--------------------------+-----------------+
[1 rows x 14 columns]

In [6]:
params = {'target':'TARGET', 'random_seed':8923,
          'class_weights':'auto','metric':'auc',
          'early_stopping_rounds':20, 'max_iterations':500,
          'step_size': 0.1,
          'max_depth': 5,
          'column_subsample': 0.6}

In [7]:
auc = 0
for train, validate in folds:
    model = gl.boosted_trees_classifier.create(train, validation=validate, verbose=False, **params)
    auc += model.evaluate(validate, metric='auc')['auc']
    print model.evaluate(validate, metric='auc')['auc']
    
print "Average:", auc/5.0

0.832100155689
0.825703819715
0.839134865427
0.850255989308
0.835867180468
Average: 0.836612402121
